In [1]:
#-----Εισαγωγή απαραίτητων πακέτων-----
from collections import defaultdict
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import networkx as nx
import matplotlib
matplotlib.use("agg")
from matplotlib import pyplot as plt

In [8]:
#-----Διαμόρφωση γραφήματος-----
# Δημιουργία ενός κενού γραφήματος
G = nx.Graph()
# Προσθήκη ακμών και κόμβων στο γράφημα
G.add_edges_from([(0,1),(0,2),(1,3),(2,3),(2,4),(3,4)])

# -----Δημιουργία του λεξικού dictionary QUBO-----
# Αρχικοποίηση του πινακα Q
Q = defaultdict(int)
# Ενημέρωση του πίνακα Q για κάθε ακμή στο γράφημα
for i, j in G.edges:
    Q[(i,i)]+= -1
    Q[(j,j)]+= -1
    Q[(i,j)]+= 2


#-----Εκτέλεση του QUBO στο QPU-----
# Ορισμός παραμέτρων στο QPU
chainstrength = 8
numruns = 10

# Εκτέλεση του QUBO στον επιλυτή από το αρχείο ρύθμισης
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               label='Example - Maximum Cut 1 ')
energies = iter(response.data())

#-----Εκτύπωση αποτελεσμάτων στον χρήστη-----
print('-' * 60)
print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
print('-' * 60)
# Επανάληψη της απόκρισης και εκτύπωση των αποτελεσμάτων
for line in response:
    # Εντοπισμός κόμβων σε κάθε σύνολο
    S0 = [k for k,v in line.items() if v == 0]
    S1 = [k for k,v in line.items() if v == 1]
    E = next(energies).energy

# Άλλος τρόπος εκτύπωσης αποτελεσμάτων, χρησιμοποιώντας το response.data()
for sample, E in response.data(fields=['sample','energy']):
    # Εντοπισμός κόμβων σε κάθε σύνολο
    S0 = [k for k,v in sample.items() if v == 0]
    S1 = [k for k,v in sample.items() if v == 1]
    # Εκτύπωση αποτελεσμάτων       
    print('{:>15s}{:>15s}{:^15s}{:^15s}'.format(str(S0),str(S1),str(E),str(int(-1*E))))

#-----Προβολή αποτελεσμάτων στον χρήστη-----
# Λήψη του βέλτιστου αποτελέσματος
# Σημείωση: Το βέλτιστο αποτέλεσμα είναι αυτό με τη χαμηλότερη ενέργεια
# Σημείωση: Ο πίνακας αντιστοίχισης (lut) αποτελεί ένα λεξικό dictionary, όπου το κλειδί είναι ο αριθμός του κόμβου
# και η τιμή είναι η ετικέτα του συνόλου. Για παράδειγμα,lut[5] = 1, υποδηλώνει ότι ο κόμβος 5 είναι στο σύνολο 1(S1.
lut = response.first.sample
#-----Ερμηνεία του βέλτιστου αποτελέσματος σε όρους κόμβων και ακμών-----
# Κόμβοι στο σύνολο 0 (S0)
S0 = [node for node in G.nodes if not lut[node]]
# Κόμβοι στο σύνολο 1 (S1)
S1 = [node for node in G.nodes if lut[node]]
# Ακμές μεταξύ των συνόλων
cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
# Ακμές εντός των συνόλων
uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

#-----Προβολή του βέλτιστου αποτελέσματος-----
# Υπολογισμός της διάταξης για τους κόμβους του γραφήματος
pos = nx.spring_layout(G)
# Καθαρισμός του τρέχοντος σχήματος
plt.clf()
#-----Σχεδίαση των κόμβων και των ακμών βάσει των συνόλων και των ακμών που τις διαχωρίζουν-----
# Κόμβοι στο σύνολο 0 (S0) με χρώμα κόκκινο  
nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='r')
# Κόμβοι στο σύνολο 1 (S1) με χρώμα κυανό  
nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='c')
# Ακμές που διασπούν τα σύνολα
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=3)
# Ακμές που συνδέουν τους κόμβους στο ίδιο σύνολο
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=3)
# Ετικέτες των κόμβων
nx.draw_networkx_labels(G, pos)
# Όνομα αρχείου για την αποθήκευση της γραφικής παράστασης 
filename = "maxcut_plot1.png"
# Αποθήκευση σχήματος σε αρχείο
plt.savefig(filename, bbox_inches='tight')
# Εκτύπωση μηνύματος με την τοποθεσία του αποθηκευμένου σχήματος
print("\nYour plot is saved to {}".format(filename))

------------------------------------------------------------
          Set 0          Set 1    Energy        Cut Size    
------------------------------------------------------------
         [0, 3]      [1, 2, 4]     -5.0             5       
         [1, 2]      [0, 3, 4]     -5.0             5       
      [1, 2, 4]         [0, 3]     -5.0             5       

Your plot is saved to maxcut_plot1.png
